In [1]:
import pandas as pd
import os

In [2]:
data_path = '\\'.join(os.getcwd().split('\\')[:-1]) + '\\data'

In [3]:
sensor_data = pd.read_csv(f'{data_path}\\background_removed_measured_ch4.csv')
emissions_data = pd.read_csv(f'{data_path}\\emission_data_processed.csv').drop_duplicates()

### Prep and pivot emissions data

In [4]:
emissions_data['start_time'] = pd.to_datetime(emissions_data.start_time)
emissions_data['end_time'] = pd.to_datetime(emissions_data.end_time)
emissions_data['start_time_seconds'] = pd.to_datetime(emissions_data.start_time).map(pd.Timestamp.timestamp)
start_time_utc_seconds = emissions_data.start_time_seconds.min()
emissions_data.start_time_seconds -= start_time_utc_seconds
emissions_data['end_time_seconds'] =   pd.to_datetime(emissions_data.end_time).map(pd.Timestamp.timestamp) - start_time_utc_seconds


In [ ]:
# Need (0,2),(2,6), but won't get it.
# emissions_data = pd.DataFrame({"start_time_seconds": [0,2,2,4,8,10], "end_time_seconds" : [6,5,4,6,10,12] })
# check next start time value, see if it lies in data point's interval. If so, split
while True:
    emissions_data = emissions_data.sort_values('start_time_seconds')
    emissions_data['shifted_start'] = emissions_data.start_time_seconds.shift(-1).fillna(1e10)
    emissions_data['shifted_start'] = emissions_data.groupby('start_time_seconds')['shifted_start'].transform('max')
    emissions_data['bool'] =  (emissions_data.shifted_start != emissions_data.start_time_seconds) & (emissions_data.shifted_start < emissions_data.end_time_seconds)
    temp1 = emissions_data[emissions_data['bool'] == True].copy()
    temp2 = temp1.copy()
    temp1['end_time_seconds'] = temp1['shifted_start']
    temp2['start_time_seconds'] = temp2['shifted_start']
    emissions_data = pd.concat([emissions_data[emissions_data['bool'] == False],temp1,temp2])
    if emissions_data['bool'].sum() == 0:
        break
#GIVEN all start points align, this works for the other side. Make sure different end - times have their respective intervals. ex: (4,6), (4,7) -> (4,6),(4,6),(6,7)
emissions_data['window_end'] = emissions_data.groupby('start_time_seconds')['end_time_seconds'].transform('min')
while True:
    if (emissions_data.window_end - emissions_data.end_time_seconds).sum() == 0:
        break
    temp = emissions_data[emissions_data.window_end != emissions_data.end_time_seconds].copy()
    temp.start_time_seconds = temp.window_end
    temp['window_end'] = temp.groupby('start_time_seconds')['end_time_seconds'].transform('min')
    emissions_data['end_time_seconds'] = emissions_data.window_end
    emissions_data = pd.concat([emissions_data,temp])
print(emissions_data.source.isna().sum())

emissions_data = emissions_data.sort_values('start_time_seconds')
emissions_data = emissions_data.groupby(['start_time_seconds','end_time_seconds']).agg(list).reset_index()
emissions_data['duration'] = emissions_data.end_time_seconds - emissions_data.start_time_seconds
emissions_data['num_active_sources'] = emissions_data.source.apply(lambda x : len(x))
print(emissions_data[emissions_data.duration > 30*60])

611


AttributeError: 'DataFrame' object has no attribute 'source'

### Prep Sensor Data

In [ ]:
sensor_data['time_stamp'] = pd.to_datetime(sensor_data.time_stamp)
sensor_data['time_stamp_seconds'] = pd.to_datetime(sensor_data.time_stamp).map(pd.Timestamp.timestamp)
sensor_data.time_stamp_seconds -= start_time_utc_seconds
print(sensor_data)


In [ ]:
emissions_data = emissions_data[['start_time_seconds','end_time_seconds','source','emission_rate']]
emissions_data.to_csv('hello.csv')
# print(emissions_data)